In [ ]:
from IPython.display import display
import spot

import sys
sys.path.append('../')
import auto
import parse
spot.setup()

'''
This notebook allows you to step through our cause-synthesis algorithm (Section 5.2 in the paper "Synthesis of Temporal Causality").
You need to specify the inputs that you would provide to the commandline in the following.
'''
limit_assumption = False
contingencies = False
system = spot.automaton('../examples/simple1/system.hoa')
effect_automaton_neg = parse.effectfile('../examples/simple1/effect.txt')
trace = parse.tracefile('../examples/simple1/trace.txt')
print(trace)
system.show()

In [ ]:
'''
As a first step, we construct the metric <=_subset and its associated automaton based on the inputs of the provided system.
'''

outputs = spot.get_synthesis_output_aps(system)
inputs = set(str(a) for a in system.ap()).difference(set(str(p) for p in outputs))

# Construct distance metric.
distance_metric = "G (True"
for i in inputs:
    distance_metric += " & (!(" + i + "_actual <-> " + i + "_close) -> !(" + i + "_actual <-> " + i + "_far))"
    if limit_assumption:
        distance_metric = "((G F !(" + i + "_actual <-> " + i + "_close)) -> (G (" + i + "_close <-> " + i + "_far))) & " + distance_metric
distance_metric += ")"
distance_automaton = spot.translate(spot.formula(distance_metric))
print(distance_metric)
distance_automaton.show()


In [ ]:
''' If desired, a contingency automaton can be computed. Otherwise we just show the system here.'''
if contingencies:
        system = auto.construct_counterfactual_automaton(system,trace)
system.show()

In [ ]:
'''Next up, we complement the effect automaton.'''

effect_automaton_neg = auto.add_suffix(spot.postprocess(effect_automaton_neg,'buchi','state-based','small','buchi'),"_close")

effect_automaton_neg.show()

In [ ]:
'''Next, we intersect effect and distance metric.'''

intersection = spot.product(distance_automaton,effect_automaton_neg)

intersection.show()

In [ ]:
''' We now construct an NBA for the system product.'''

inner_product = spot.product(auto.add_suffix(system,"_close"),intersection)

inner_product.show()

In [ ]:
''' Project APs suffixed with _close away existentially.'''

after_projection = auto.project_existentially(inner_product,[str(a) + "_close" for a in system.ap()])

after_projection.show()

In [ ]:
''' Final complementation, intersection with actual trace and AP manipulation to obtain the desired cause automaton.'''

# Central complementation.
intermediate_result = spot.complement(after_projection)

# Intersect with actual trace and project away APs
actual_trace = auto.add_suffix(trace.as_automaton(),"_actual")
actual_result = spot.product(intermediate_result,actual_trace)
actual_projection = auto.project_existentially(actual_result,[str(a) + "_actual" for a in system.ap()])

intermediate_result = spot.postprocess(actual_projection,'buchi','state-based','small','high')

# Map APs back to inputs by removing the dummy suffix.
result = auto.remove_suffix(intermediate_result,"_far")

result.show()